# TagTouch

To compute and run trajectories between to points with the Dingo-Gen3 lite and to detect markers.

In [ ]:
# Insérez ici le numéro de votre équipe
# (en) Insert here your team number
VOTRE_NUMERO_EQUIPE = 0
# modifier cette variable selon que vous contrôler la simulation ou le robot réel
# (en) change this variable according to whether you control the simulation or the real robot
SIM = True

## Importation des paquets requis

In [ ]:
import sys; sys.path.append('/opt/ros/noetic/lib/python3/dist-packages/')
sys.path.append('/home/admin_mec/mobile_manip_ws/devel/lib/python3/dist-packages/')
from os import environ
if SIM:
    #environ['ROS_MASTER_URI'] = "http://localhost:1137{}/".format(VOTRE_NUMERO_EQUIPE)
    environ['ROS_MASTER_URI'] = "http://localhost:11311/".format(VOTRE_NUMERO_EQUIPE)
else:
    environ['ROS_MASTER_URI'] = "http://cpr-ets05-01:11311/"
    environ['ROS_IP'] = "192.168.0.81"
import numpy as np
import math
from scipy.spatial.transform import Rotation as R
from lab_utils.plan_utils import *
import rospy
from jackal_msgs.msg import Drive
from nav_msgs.msg import Odometry, MapMetaData
from std_msgs.msg import Float64
from geometry_msgs.msg import Pose, PoseWithCovarianceStamped
from geometry_msgs.msg import Twist as GT
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from apriltag_ros.msg import AprilTagDetectionArray
from mobile_manip.srv import ReachName, GetValues, ReachValues

In [ ]:
# Créer et démarrer un nouveau noeud
# (en) Create and start a new node
rospy.init_node('mobile_manip_navigation', anonymous=True)

## Utility functions

In [ ]:
# Fonction pour le calcul de l'orientation à partir d'un quaternion
# (en) Function for calculating the orientation from a quaternion
def get_heading_from_quaternion(q):
    r = R.from_quat([q.x, q.y, q.z, q.w])
    angles = r.as_euler('xyz', degrees=False)
    return angles[2]

def get_quaternion_from_heading(z):
    r = R.from_euler('z', z)
    return r.as_quat()

# Returns the smallest difference between to angles in the range [-pi, pi]
def wraptopi(angle):
    xwrap=np.remainder(angle, 2*np.pi)
    if np.abs(xwrap)>np.pi:
        xwrap -= 2*np.pi * np.sign(xwrap)
    return xwrap
def angle_diff(a, b):
    diff = a - b
    if diff > math.pi: diff  -= 2*math.pi
    if diff < -math.pi: diff += 2*math.pi
    return diff

## Importation de la carte (format png)

In [ ]:
start = Point(1380, 1130)
goal = Point(1590, 1545)
fig, ax = plt.subplots(figsize=(18,10))
image = mpimg.imread("a2230_map_closed_fliped.png")
plt.imshow(image, origin="lower")
ax.add_artist(plt.Circle((start.x, start.y), 6, color='r'))
ax.add_artist(plt.Circle((goal.x, goal.y), 6, color='y'))
plt.show()

## Création des 'callbacks'

Chaque fonction est associée à un sujet ROS et enregistre en continue sont contenu dans une variable globale.

(en) Each function is associated with a ROS topic and continuously records its content in a global variable.

In [ ]:
# map and position callbacks
origin = Point(0, 0)
map_resolution = 0
amcl_origin = { 'x': origin.x, 'y': origin.y, 'theta': 0.0}
def map_callback(msg):
    global origin, amcl_origin, map_resolution
    map_resolution = msg.resolution
    origin.x = np.abs(msg.origin.position.x) # en m
    origin.y = np.abs(msg.origin.position.y) # en m
    amcl_origin = { 'x': origin.x, 'y': origin.y, 'theta': 0.0} # en m

amcl_pose = Pose()
def amcl_callback(msg):
    global amcl_pose
    x = msg.pose.pose.position.x
    y = msg.pose.pose.position.y
    theta = get_heading_from_quaternion(msg.pose.pose.orientation)
    amcl_pose.position.x = x * math.cos(amcl_origin['theta']) - y * math.sin(amcl_origin['theta']) + amcl_origin['x']
    amcl_pose.position.y = x * math.sin(amcl_origin['theta']) + y * math.cos(amcl_origin['theta']) + amcl_origin['y']
    quat = get_quaternion_from_heading(theta + amcl_origin['theta'])
    amcl_pose.orientation.x = quat[0]
    amcl_pose.orientation.y = quat[1]
    amcl_pose.orientation.z = quat[2]
    amcl_pose.orientation.w = quat[3]

In [ ]:
# AprilTag subscriber callback
tag_array = AprilTagDetectionArray()
def tag_callback(msg):
    global tag_array
    tag_array  = msg

In [ ]:
# ROS subscribers et publishers
cmd_drive_pub = rospy.Publisher('/mobile_manip/dingo_velocity_controller/cmd_drive', Drive, queue_size=1)
cmd_vel_pub = rospy.Publisher('/mobile_manip/cmd_vel', GT, queue_size=1)
map_sub = rospy.Subscriber('/map_metadata', MapMetaData, map_callback) # Important before amcl_sub to get map_data
amcl_sub = rospy.Subscriber('/amcl_pose', PoseWithCovarianceStamped, amcl_callback)
tag_sub = rospy.Subscriber('/tag_detections', AprilTagDetectionArray, tag_callback)

## Contrôleur du mouvement

In [ ]:
interwheel_distance = 0.3765
left_wheel_radius = 0.098
right_wheel_radius = 0.098
def move_robot(linear, angular):
    vel_left  = (linear - angular * interwheel_distance / 2.0) / left_wheel_radius
    vel_right = (linear + angular * interwheel_distance / 2.0) / right_wheel_radius

    if SIM:
        # Envoi des commandes au roues par topic ROS
        # (en) Sending commands to the wheels via ROS topic
        cmd_drive_msg = Drive()
        cmd_drive_msg.drivers[0] = vel_left
        cmd_drive_msg.drivers[1] = vel_right
        cmd_drive_pub.publish(cmd_drive_msg)
    else:
        # Envoi des commandes au controlleur du robot
        # (en) Sending commands to the robot controller
        cmd_vel_msg = GT()
        cmd_vel_msg.linear.x = linear
        cmd_vel_msg.angular.z = angular
        cmd_vel_pub.publish(cmd_vel_msg)

## Trajectoire avec A*

In [ ]:
import time
time.sleep(0.1)
start = Point(int(amcl_pose.position.x/map_resolution), int(amcl_pose.position.y/map_resolution))
print(start.x,start.y, get_heading_from_quaternion(amcl_pose.orientation))

In [ ]:
from lab_utils.astart import AStarPlanner
# Create the points for the algorithm
start = Point(int(amcl_pose.position.x/map_resolution), int(amcl_pose.position.y/map_resolution))
map_img = 1-np.array(image[:,:,1])
mat_map = map_img
map = BMPMap(width=map_img.shape[1], height=map_img.shape[0], mat=mat_map)
astarPlanner = AStarPlanner(map=map, step_size=6, collision_radius=12, heuristic_dist='Euclidean')
astarPlanner.plan(start=start, target=goal)

In [ ]:
fig = plt.figure(figsize=(16,8))
ax2 = fig.add_subplot(1, 1, 1)
plt.imshow(image, origin='lower')
ax2.add_artist(plt.Circle((start.x, start.y), 10, color='r'))
ax2.add_artist(plt.Circle((goal.x, goal.y), 10, color='y'))
for i in range(len(astarPlanner.finalPath)-1):
    pt = astarPlanner.finalPath[i].tuple()
    ax2.add_artist(plt.Circle((pt[0], pt[1]), 5, color='g'))
plt.show()

## Suivre la trajectoire (Attention le robot va bouger!)
Gardez vos mains sur la télécommande

In [ ]:
current_wp = 0
while current_wp < len(astarPlanner.finalPath):
    target_x, target_y = astarPlanner.finalPath[current_wp].tuple()

    target_x *= map_resolution
    target_y *= map_resolution

    #print("From {}, {} to {}, {}".format(pose_msg.position.x, pose_msg.position.y, target_x, target_y))

    theta_target = math.atan2(target_y - amcl_pose.position.y, target_x - amcl_pose.position.x)
    theta = get_heading_from_quaternion(amcl_pose.orientation)

    dist = math.sqrt((target_x-amcl_pose.position.x)**2 + (target_y-amcl_pose.position.y)**2)
    dtheta = angle_diff(theta_target, theta)

    #print(dist, dtheta)

    if dist > 0.4:
        if math.fabs(dtheta) > 0.785:
            linear = 0.0
        else:
            linear = 0.3

        angular = min(dtheta, 0.7) if dtheta >= 0 else max(dtheta, -0.7)
        move_robot(linear, angular)
    else:
        current_wp += 1

move_robot(0, 0)

### Relever le bras pour libérer le chemin (Lift the arm to clear the path)

## Déplacement de la base vers le tag (Move the base to the tag)

In [ ]:
from scipy.spatial.transform import Rotation as R
dist = 1.0
if SIM:
    T_cam2base = np.array([[0.16], [0.04], [0.463]])
else:
    T_cam2base = np.array([[0.0], [0.05], [-0.137]])


## Déplacement du bras vers le tag (Move the arm to the tag)

In [ ]:
# Touche le Tag
# (en) Touch the Tag


## Déplacement du bras vers la poignée (Move the arm to the handle)

In [ ]:
# Ouvre la pince
# (en) Open the gripper


In [ ]:
# Déplace vers la poignée
# (en) Move to the handle


In [ ]:
# Ferme la pince
# (en) Close the gripper


In [ ]:
# Pour obtenir les coordonnées actuelles de la pince
# (en) To get the current coordinates of the gripper
